Setup

In [9]:
from google.colab import drive
import pandas as pd

'''
Test data recorded from Matt's private BeefBeef server

Description of training data from University of Minnesota:
  These files contain 1,000,209 anonymous ratings of approximately 3,900 movies
  made by 6,040 MovieLens users who joined MovieLens in 2000.
'''
drive.mount('/content/drive')
test_path = '/content/drive/My Drive/matt_beefbeef_server/out.csv'
train_path = '/content/drive/My Drive/matt_beefbeef_server/ratings.csv'
metadata_path = '/content/drive/My Drive/matt_beefbeef_server/movies_metadata.csv'

test = pd.read_csv(test_path)
metadata = pd.read_csv(metadata_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-9-a418bb45270f>:17: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata = pd.read_csv(metadata_path)


In [2]:
# Load big file in separately
train = pd.read_csv(train_path)


Examine & prepare test data

In [3]:
print('BEEFBEEF DATA\n-------------\n')

# Get table size
def tablesize():
  print(f'Rows:  {test.shape[0]}', f'\nColumns:  {test.shape[1]}')


# Find date ranges in data
def timeinterval():
  test['DateCreated'] = pd.to_datetime(test['DateCreated'])

  earliest = (test['DateCreated'].min()).strftime('%d %B %Y')
  latest = (test['DateCreated'].max()).strftime('%d %B %Y')

  print(f'\nShowing data from: \n>  {earliest} \nto \n>  {latest}')


# See data features
def column_names():
  print('\nColumn names:')
  for column in test.columns:
    print('>  ', column)


# Basic descriptive stuff
def user_movie_unique():
  print('\nUsers:  ', test['UserId'].nunique())
  print('Movies:', test['ItemName'].nunique())


tablesize()
timeinterval()
column_names()
user_movie_unique()

test = test.iloc[:, [1, 4]]
test.head()


BEEFBEEF DATA
-------------

Rows:  2099 
Columns:  9

Showing data from: 
>  15 May 2023 
to 
>  21 October 2023

Column names:
>   DateCreated
>   UserId
>   ItemId
>   ItemType
>   ItemName
>   PlaybackMethod
>   ClientName
>   DeviceName
>   PlayDuration

Users:   13
Movies: 1066


,UserId,ItemName
0,a31bac6c6edf406b9d2031d8de86f0d0,Bottle Rocket
1,a31bac6c6edf406b9d2031d8de86f0d0,Bottle Rocket
2,a31bac6c6edf406b9d2031d8de86f0d0,Bottle Rocket
3,3687fd7ac2444e43853b01d6f6540ad9,Ghost in the Shell
4,3687fd7ac2444e43853b01d6f6540ad9,Ghost in the Shell


Examine train data

In [4]:
train.head()


,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


In [10]:
metadata = metadata.iloc[:, [5, 20]]
metadata.head()


,id,title
0,862,Toy Story
1,8844,Jumanji
2,15602,Grumpier Old Men
3,31357,Waiting to Exhale
4,11862,Father of the Bride Part II


In [6]:
# Re-format variables to allow matching
metadata['id'] = metadata['id'].astype(str)
train['movieId'] = train['movieId'].astype(str)

# Table join on movie id
merged_data = pd.merge(train, metadata, left_on='movieId', right_on='id', how='left')


   userId movieId  rating   timestamp    id                 title
0       1     110     1.0  1425941529   110     Three Colors: Red
1       1     147     4.5  1425942435   147         The 400 Blows
2       1     858     5.0  1425941523   858  Sleepless in Seattle
3       1    1221     5.0  1425941546   NaN                   NaN
4       1    1246     5.0  1425941556  1246          Rocky Balboa


In [7]:
merged_data.head()


,userId,movieId,rating,timestamp,id,title
0,1,110,1.0,1425941529,110,Three Colors: Red
1,1,147,4.5,1425942435,147,The 400 Blows
2,1,858,5.0,1425941523,858,Sleepless in Seattle
3,1,1221,5.0,1425941546,NaN,NaN
4,1,1246,5.0,1425941556,1246,Rocky Balboa


In [8]:
# Big data loss...
missing_titles = merged_data['title'].isna().sum()
print("Number of missing values in the 'title' column:", missing_titles)

non_missing_titles = merged_data['title'].notna().sum()
print("Number of non-missing values in the 'title' column:", non_missing_titles)

total_values = len(merged_data['title'])
missing_values = merged_data['title'].isna().sum()
percentage_missing = (missing_values / total_values) * 100
print("Percentage of missing values in the 'title' column:", percentage_missing)


Number of missing values in the 'title' column: 14587721
Number of non-missing values in the 'title' column: 11437637
Percentage of missing values in the 'title' column: 56.051951331466796


In [11]:
# Something wrong with join
missing_titles_metadata = metadata['title'].isna().sum()
print("Number of missing movie titles in the original metadata table:", missing_titles_metadata)


Number of missing movie titles in the original metadata table: 6


\(solution in-progress)